In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))

import shutil
import datetime
import itertools
import numpy as np
import pandas as pd

from library import common as cm
from library import regression_aux as raux

In [ ]:
%run setup.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_SEED)

In [ ]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

In [ ]:
print(f'The number of samples after loading is : {df.shape[0]}')

regr_dir = res_dir + 'Regr_Bucket/'
print(f'Results are saved at:\n{regr_dir}')

In [ ]:
if os.path.exists(regr_dir):
    shutil.rmtree(regr_dir)
os.makedirs(regr_dir)
shutil.copy('setup.py', regr_dir);

In [ ]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')

##### Zero Hedge

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}No_Hedge/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    zero = np.array([0.]*len(df))
    zero = pd.Series(zero, index=df.index)

    cm.store_pnl(df, zero, pnl_path)
    save_tables(None, sub_regr_dir)

##### BS-Delta

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}BS_Benchmark/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    cm.store_pnl(df, df['delta_bs'], pnl_path)
    save_tables(None, sub_regr_dir)

##### Fixed

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Fixed_Constants/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    bl_c = df['cp_int'] == 0
    delta = 0.9 * df['delta_bs']
    delta[~bl_c] = 1.1 * df.loc[~bl_c, 'delta_bs']
    
    cm.store_pnl(df, delta, pnl_path)
    save_tables(None, sub_regr_dir)

In [ ]:
""" 
The bucket will be like 0, 1, 0, 1. The first two are calls, and the last two are puts
0 means less than median, and 1 means larger than median.
"""
bl_c = df['cp_int'] == 0
call_median = df.loc[bl_c, 'M0'].median()
put_median = df.loc[~bl_c, 'M0'].median()

bl_call_median = df['M0'] > call_median
df['bucket'] = 0
df.loc[bl_c & bl_call_median, 'bucket'] = 1

bl_put_median = df['M0'] > put_median
df.loc[(~bl_c) & bl_put_median, 'bucket'] = 1

##### Regressions

In [ ]:
all_regressions = [('Delta_only', ['delta_bs'], False),
    ('Delta_Vega', ['delta_bs', 'vega_n'], False),
    ('Delta_Gamma', ['delta_bs', 'gamma_n'], False),
    ('Delta_Vanna', ['delta_bs', 'vanna_n'], False),
    ('Delta_Gamma_Vanna', ['delta_bs', 'gamma_n', 'vanna_n'], False),
    ('Delta_Vega_Gamma', ['delta_bs', 'vega_n', 'gamma_n'], False),
    ('Delta_Vega_Vanna', ['delta_bs', 'vega_n', 'vanna_n'], False),
    ('Delta_Vega_Gamma_Va', ['delta_bs', 'vega_n', 'gamma_n', 'vanna_n'], False),
    ('Vega_only', ['vega_n'], True),
     ('Gamma_only', ['gamma_n'], True),
     ('Vanna_only', ['vanna_n'], True),
     ('Bias', ['bias'], True),
     ('Hull_White', ['vega_s', 'delta_vega_s', 'delta2_vega_s'], True),
    ('Hull_White_relaxed', ['delta_bs', 'vega_s', 'delta_vega_s', 'delta2_vega_s'], False)
]

In [ ]:
if not PERMUTE:
    df['bias'] = 1.
    for a, features, delta_1 in all_regressions:
        sub_regr_dir = f'{regr_dir}/{a}/'
        kwargs = {'vix': VIX, 
              'features': features, 
              'max_period': max_period, 
              'sub_res': f'{sub_regr_dir}pnl/',
              'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
              'df': df,
              'delta_coeff_1': delta_1,
              'agg_side': False,
              'leverage': False,
                 'bucket': True}
        res = raux.run_store_lin(**kwargs)
        save_tables(res, dirs=sub_regr_dir)

In [ ]:
# Cols are different data sets, rows are different experiments.
cols = [
    'FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_MINTAU=0_Permute=False_VIX=False_MISSING=replace_missing_WINDOW=180D_KickTmrZeroVolume=False',
    'FREQ=2D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_MINTAU=0_Permute=False_VIX=False_MISSING=replace_missing_WINDOW=180D_KickTmrZeroVolume=False',
       ]
rows = ['Regr_Bucket/No_Hedge', 'Regr_Bucket/BS_Benchmark', 'Regr_Bucket/Fixed_Constants',
        'Regr_Bucket/Delta_only', 'Regr_Bucket/Vega_only', 'Regr_Bucket/Gamma_only','Regr_Bucket/Vanna_only',  
        'Regr_Bucket/Bias',
        'Regr_Bucket/Delta_Gamma', 'Regr_Bucket/Delta_Vega',   'Regr_Bucket/Delta_Vanna',
        'Regr_Bucket/Delta_Vega_Gamma', 'Regr_Bucket/Delta_Vega_Vanna', 'Regr_Bucket/Delta_Gamma_Vanna',   
        'Regr_Bucket/Delta_Vega_Gamma_Va', 
        'Regr_Bucket/Hull_White', 'Regr_Bucket/Hull_White_relaxed',
        #'Network/Normal_Feature', 'Network/Delta_Vega', 'Network/Delta_Vega_Vanna',
       #'Network/Normal_Feature_CDF', 'Network/Spot_Strike', 'Network/Spot_Strike_2'
       ]

In [ ]:
df_dirs = pd.DataFrame(columns=cols, index=rows)
for x, y in list(itertools.product(rows, cols)):
    df_dirs.loc[x, y] = f'{DATA_DIR}Result/{y}/{x}/'

In [ ]:
def run_save_tables(dirs, agg, measure, op_type):
    inspector = cm.Inspector()
    df_res = inspector.evalPnls(df_dirs, aggregating=aggregating, measure=measure, op_type=op_type)
    if op_type == None:
        file = f'{DATA_DIR}Result/bucket_call_and_put_{measure}.csv'
    else:
        file = f'{DATA_DIR}Result/bucket_{op_type}_{measure}.csv'
    with open(file, 'w+') as f:
        f.write(f'{datetime.datetime.now()}\n{op_type}_{measure}\n')
    (df_res).to_csv(file, mode='a') 
    return df_res

In [ ]:
aggregating = 'mean'
measure = 'mse'

In [ ]:
run_save_tables(df_dirs, agg=aggregating, measure=measure, op_type=None)